In [1]:
import pandas as pd
import numpy as np

In [2]:
class ElemPala:
    """Clase que te permite obtener los parámetros diferenciales (Levantamiento, Arrastre, Torque)
       de cada elemento de una pala de un autogiro o helicóptero, se puede obtener el número
       de revoluciones necesario para entrar en autorotación cuando el torque total es cero
    """
    
    def __init__(self, omega):
        self.omega = omega
        self.W = 450*9.81
        self.D = 8.4; self.cuerda = 0.24; self.a = 5.6; self.Cdo = 0.007
        self.l_efec = 3.68
        self.dr = 0.08
        self.num_elem = self.l_efec/self.dr
        self.r_0 = self.D/2 - 3.68
        self.r = []
        self.alpha_fb = [] # alphas de los elementos de la pala que avanza
        self.alpha_bb = [] # alphas de los elementos de la para que retrocede
        self.VR_fb = []; self.VR_bb = []
        self.tetha = 0     # ángulo de paso de la pala
        self.T = self.W/np.cos(np.deg2rad(30))
        self.Voo = 2 * (self.T / (2 * 1.225 * np.pi * (1/4) * (self.D)**2))**0.5
        self.L_fb = []; self.L_bb = []; self.D_fb = []; self.D_bb = []
        self.Q_fb = []; self.Q_bb = []
        self.DF = pd.DataFrame({'Omega [rpm]': [0], 'Q total': [0]})
        self.Q = []
        
    def radios(self):
        """
        Método para obtener el radio correspondiente a cada elemento
        Args: 
            None
        Output: 
            Lista de la posición del radio para cada elemento
        """
        self.r = [round(r,2) for r in np.arange(self.r_0 + self.dr, self.l_efec, self.dr+0.001)]
        
    def alpha(self):
        """
        Método que nos permite obtener el ángulo de ataque de cada elemento
        Args: 
            None
        Output: 
            Ángulo de ataque de cada elemento para un cierto rpm de ambas palas
        """
        i = 0
        for r in self.r:
            self.VR_fb.append(self.Voo * np.cos(np.deg2rad(30)) + self.omega*r/60*(2*np.pi/1))
            self.alpha_fb.append(np.arctan((self.Voo * np.sin(np.deg2rad(30)))/self.VR_fb[i]) + self.tetha)
            self.VR_bb.append(abs(self.Voo * np.cos(np.deg2rad(30)) - self.omega*r/60*(2*np.pi/1)))
            self.alpha_bb.append(np.arctan((self.Voo * np.sin(np.deg2rad(30)))/self.VR_bb[i]) + self.tetha)
            i += 1
    
    def delta_L(self):
        """Método para calcular el delta de Levantamiento en cada elemento
        """
        i = 0
        for VR in self.VR_fb:
            self.L_fb.append(0.5*1.225*(VR**2)*self.cuerda*self.a*self.alpha_fb[i]*self.dr)
            i += 1
            
        i = 0
        for VR in self.VR_bb:
            self.L_bb.append(0.5*1.225*(VR**2)*self.cuerda*self.a*self.alpha_bb[i]*self.dr)
            i += 1
            
    def delta_D(self):
        """Método para calcular el delta de Levantamiento en cada elemento
        """
        i = 0
        for VR in self.VR_fb:
            self.D_fb.append(0.5*1.225*(VR**2)*self.cuerda*self.Cdo*self.dr)
            i += 1
            
        i = 0
        for VR in self.VR_bb:
            self.D_bb.append(0.5*1.225*(VR**2)*self.cuerda*self.Cdo*self.dr)
            i += 1
        
    
    def torque(self):
        """Método para calcular el torque total de ambas palas
        """
        for i in range(len(self.VR_fb)):
            self.Q_fb.append((self.D_fb[i] - self.alpha_fb[i]*self.L_fb[i])*self.r[i]*self.dr)
            self.Q_bb.append((self.D_bb[i] - self.alpha_bb[i]*self.L_bb[i])*self.r[i]*self.dr)
        
        return sum(self.Q_fb) + sum(self.Q_bb)
    
    

In [4]:
def data_frame(omega_list):
        """Función para construir un Data Frame de pandas para visualizar los cálculos con diferentes rpm
        """
        global DF
        i = 0
        Q = []
        for j in range(len(omega_list)):
            AG = ElemPala(omega_list[i])
            AG.radios()
            AG.alpha()
            AG.delta_L()
            AG.delta_D()
            Q.append(AG.torque())
            i += 1
            
        DF = pd.DataFrame({'Omega [rpm]': omega_list, 'Q total': Q})

In [5]:
omega_lst = [omega for omega in range(3600, 4000, 50)]
data_frame(omega_lst)

### DF con error en omega

In [7]:
DF

,Omega [rpm],Q total
0,3600,-4.850387
1,3650,-4.070631
2,3700,-3.280003
3,3750,-2.478510
4,3800,-1.666156
5,3850,-0.842945
6,3900,-0.008882
7,3950,0.836031


### DF sin error en omega

In [11]:
omega_lst = [omega for omega in range(500, 1000, 50)]
data_frame(omega_lst)

In [12]:
DF

,Omega [rpm],Q total
0,500,-11.491437
1,550,-7.038979
2,600,-2.156313
3,650,3.155106
4,700,8.894368
5,750,15.060882
6,800,21.654247
7,850,28.674185
8,900,36.120497
9,950,43.993038


In [51]:
AG1 = ElemPala(620.78)
AG1.radios()
AG1.alpha()
AG1.delta_L()
AG1.delta_D()
AG1.torque()

-0.0008929127681880367

In [52]:
AG1.T

5097.425526675205

In [53]:
delta_T = map(lambda x, y: x/np.cos(np.deg2rad(30)) + y/np.cos(np.deg2rad(30)), AG1.L_fb, AG1.L_bb)

In [54]:
T_total = list(delta_T)
sum(T_total)

5048.102885851675